In [1]:
import torch
import tifffile
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  
import os
from torch.cuda.amp import GradScaler, autocast
from Models.LPTISR import LPTISR

C:\Users\Dell\.conda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
head_dir = r"F:\Opensource\BioTISR\MT\Testing"
Raw_path = head_dir + r'\TestNoisy'
all_imgs_path = glob.glob(Raw_path + '\\'+'*.tif')
num = len(all_imgs_path)
Endstr ='.tif'
device = 'cuda'
save_path = head_dir + '\\'+'output'
if not os.path.exists(save_path):  
    os.mkdir(save_path)

In [20]:
model_label = r'LPTISR'
model =  LPTISR(2,7,False)
model.load_state_dict(torch.load(r'TrainedModels\BioTISR\MT\best_model.pth'))
model_label = model_label
save_path = head_dir + '\\'+'output'+'\\'+model_label
if not os.path.exists(save_path):  
    os.mkdir(save_path)

pbar = tqdm(total=num, desc="Processing images")
for i in range(num):     
    numstr = np.array2string(np.array(i+1))
    I = tifffile.imread(Raw_path+ '\\' +numstr+Endstr)  # 20*51*512
    I = np.array(I)
    d1,_,_ = I.shape
    I = (I - I.min()) / (I.max() - I.min())
    model = model.to(device)
    for frame in range(d1-6):
        
        im = torch.tensor(I[frame:frame+7,...], dtype=torch.float16).unsqueeze(0).unsqueeze(2).to(device)
        
        with torch.no_grad():
            model.eval()
            with autocast():
                output = model(im.to(device))
            output = output.to('cpu')
            output = output.detach()
            imde = output.squeeze(0) #7*1024*1024
            if frame==0:
                imde = imde[0:4,...]
            elif frame==(d1-7):
                imde = imde[3:7,...]
            else:
                imde = imde[3,...].unsqueeze(0)
        if frame == 0:
            out_batch = imde
        else:
            out_batch = torch.cat((out_batch,imde),0)
        
            
    imde = np.array(out_batch)*65535
        #imde = (2**16-1)*(imde-imde.min())/(imde.max()-imde.min())
    imde = imde.astype(np.uint16)
    
    imde[imde<0] = 0
    tifffile.imwrite(save_path+ '\\' +numstr+model_label+Endstr,imde)
    pbar.update(1)
pbar.close()

drop path: Identity()
drop path: Identity()
drop path: Identity()


Processing images: 100%|██████████| 30/30 [03:00<00:00,  6.01s/it]


In [21]:
# Evaluation
import glob
import numpy as np
import tifffile
from scipy.optimize import leastsq
import os
import math
import numexpr
import cv2
import torch

In [22]:
from scipy.optimize import leastsq

def linear_error(p,gt,pre):
    return p[0]*pre+p[1]-gt
    
def linear_trans(gt,pre):
    p0 = [1,0]
    result = leastsq(linear_error,p0,args=(gt.ravel(),pre.ravel()))
    a,b = result[0]
    return a*pre+b

def calc_psnr(sr, hr, scale=3, rgb_range=255, dataset=None):
    sr = linear_trans(hr, sr)
    diff = (sr - hr)
    mse = np.mean((diff) ** 2)
    return -10 * math.log10(mse)

In [23]:
GT_dir = head_dir + r'\\TestGT\\'
Output_dir = head_dir + r'\\output\\LPTISR\\'
PSNR_list = []

num = 30
for i in range(num):
    print(i)
    numstr = np.array2string(np.array(i+1))
    GT = tifffile.imread(os.path.join(GT_dir,str(i+1)+'.tif'))
    GT = np.array(GT)
    GT = GT/GT.max()
    for tt in range(20):
        GT[tt,...] = cv2.GaussianBlur(GT[tt,...], (3,3), 0.8)
    GT = GT/GT.max()   
    Output = tifffile.imread(os.path.join(Output_dir,str(i+1)+'LPTISR.tif'))/65535

    for tt in range(8,10):
        GT_tmp = GT[tt,...]
        Output_tmp = Output[tt,...]
        PSNR_list.append(calc_psnr(Output_tmp,GT_tmp))
PSNR_list = np.array(PSNR_list)
print(PSNR_list.mean())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
29.891276055124244
